In [44]:
import numpy as np 
import pandas as pd 
import gensim
import nltk
from nltk import FreqDist
from nltk import word_tokenize
from nltk import ConditionalFreqDist

In [5]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [11]:
parameter_a = 5*0.0001

## Get our word vectors

In [4]:
word_vectors = KeyedVectors.load_word2vec_format('../data/frWac_non_lem_no_postag_no_phrase_500_skip_cut100.bin', binary=True)

In [8]:
word_vectors.most_similar(positive=['femme', 'roi'], negative=['homme'])

[('reine', 0.6213041543960571),
 ('régente', 0.5884515047073364),
 ('épousa', 0.5733684301376343),
 ('isabeau', 0.564530074596405),
 ('duc', 0.5508047342300415),
 ('épouse', 0.5422359704971313),
 ('reine-mère', 0.5368833541870117),
 ('fille', 0.534365177154541),
 ('trône', 0.5177679061889648),
 ('princesse', 0.5169864892959595)]

In [10]:
word_vectors.get_vector("redfne")

KeyError: "word 'redfne' not in vocabulary"

In [37]:
## Read the document
text = ""
f = open('../data/random_french_questions.txt', 'r')
document = f.read()
document = document.replace("\n", " ")


In [49]:
# Sentences to vectorize
f = open('../data/random_french_questions.txt', 'r')
sentences = f.readlines()
sentences = [i[0:-1] for i in sentences]
sentences

["y aurait-il un coach de disponible pour m'aider dans mon cas?",
 'Tu veux partir un squad?',
 'Comment démarrer un Squad ?',
 "Salut, sur one note, j'ai un dossier complet qui a disparu.  Comment faire pour le retrouver ?",
 "Très intéressant. J'aimerais y a ssister en personne. A quelle endroit aura lieu la présentation ?",
 "est-ce normal que nous avons reçu aujourd'hui 400 anciennes cartes au transit 0262-1 ?",
 "j'ai perdu mes écouteurs jabra.  est-ce possible de s'en procurer des nouveaux?",
 "j'aimerais disposer de clé VPN que j'ai en ma possession d'ancien employés.  Comment doit je procéder?",
 " Si un employé a un gestionnaire matriciel, est-ce que celui-ci recevra les demandes d'approbation?",
 'Candidats pour notre promo! Est-ce qu’une de vous peut communiquer avec eux?',
 "comment est ce qu'on indexe les journées d'études dans le module temps ?",
 'groupe, comment je fais pour aller su sharepoint?',
 'Es-tu déjà inscrit à un groupe de discussion culture?',
 "Une employée 

In [39]:
tokens = word_tokenize(document)
tokens

['y',
 'aurait-il',
 'un',
 'coach',
 'de',
 'disponible',
 'pour',
 "m'aider",
 'dans',
 'mon',
 'cas',
 '?',
 'Tu',
 'veux',
 'partir',
 'un',
 'squad',
 '?',
 'Comment',
 'démarrer',
 'un',
 'Squad',
 '?',
 'Salut',
 ',',
 'sur',
 'one',
 'note',
 ',',
 "j'ai",
 'un',
 'dossier',
 'complet',
 'qui',
 'a',
 'disparu',
 '.',
 'Comment',
 'faire',
 'pour',
 'le',
 'retrouver',
 '?',
 'Très',
 'intéressant',
 '.',
 "J'aimerais",
 'y',
 'a',
 'ssister',
 'en',
 'personne',
 '.',
 'A',
 'quelle',
 'endroit',
 'aura',
 'lieu',
 'la',
 'présentation',
 '?',
 'est-ce',
 'normal',
 'que',
 'nous',
 'avons',
 'reçu',
 "aujourd'hui",
 '400',
 'anciennes',
 'cartes',
 'au',
 'transit',
 '0262-1',
 '?',
 "j'ai",
 'perdu',
 'mes',
 'écouteurs',
 'jabra',
 '.',
 'est-ce',
 'possible',
 'de',
 "s'en",
 'procurer',
 'des',
 'nouveaux',
 '?',
 "j'aimerais",
 'disposer',
 'de',
 'clé',
 'VPN',
 'que',
 "j'ai",
 'en',
 'ma',
 'possession',
 "d'ancien",
 'employés',
 '.',
 'Comment',
 'doit',
 'je',
 'pr

In [47]:
frequencies= FreqDist(tokens)

In [46]:
frequencies.freq("communs")

0.0006242197253433209

In [75]:
## First vectors 
def sentence_first_vector(sentence, a, word_vectors, frequencies) : 
    tokens = word_tokenize(sentence)
    counter = 0 
    vector = [0]*500
    for i in tokens : 
        if i in word_vectors.vocab : 
            counter +=1
            weight = a/(a+frequencies.freq(i))
            word_embedding = word_vectors.get_vector(i).tolist() 
            for i in range(500): 
                vector[i] += word_embedding[i] * weight
    vector = [x / counter for x in vector]
    return vector
        

In [61]:
len(sentence_first_vector(sentences[1], parameter_a, word_vectors, frequencies))

500

In [81]:
sentences.pop(46)

'Connaissiez-vous Draw.io ?'

'Connaissiez-vous Draw.io ?'

In [82]:
sentence_matrix=[]
for i in range(len(sentences)) : 
    sentence_matrix.append(sentence_first_vector(sentences[i], parameter_a, word_vectors, frequencies) )


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [83]:
sentences_matrix= [sentence_first_vector(i, parameter_a, word_vectors, frequencies) for i in sentences]

In [89]:
sentences_array = np.array(sentences_matrix)
sentences_array= sentences_array.transpose()

In [90]:
from sklearn.utils.extmath import randomized_svd
U, S, Vt = randomized_svd(sentences_array, n_components=1)

In [91]:
U.shape

(500, 1)

In [92]:
sentences_matrix[0] = sentences_matrix[0] - U.dot(U.transpose()).dot(np.array(sentences_matrix[0]))

In [94]:
len(sentences_matrix[0])

500